In [292]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import scipy.stats as sc
from scipy import signal
from scipy import ndimage
from sklearn import datasets
from sklearn import svm
from sklearn import metrics
import sklearn.model_selection
import tqdm

In [299]:
PIC_SIZE=64
N_TRAIN=70
N_TEST=30

In [300]:
data = datasets.load_digits()

In [301]:
X = data['images']
Y = data['target']

In [303]:
ref_dig = np.random.choice(data['target_names'])
ref_dig

2

In [312]:
ref_ims = c
other_ims = X[Y!=ref_dig]
# other_dig = Y[Y!=ref_dig]

Xt=ref_ims[np.random.choice(ref_ims.shape[0],
                            size=100, replace=False)]
Xt_train = Xt[:N_TRAIN].reshape(N_TRAIN, PIC_SIZE)
Xt_test = Xt[N_TRAIN:].reshape(N_TEST, PIC_SIZE)
Yt_train=np.ones((N_TRAIN))
Yt_test=np.ones((N_TEST))

Xf=other_ims[np.random.choice(other_ims.shape[0],
                            size=100, replace=False)]
Xf_train = Xf[:N_TRAIN].reshape(N_TRAIN, PIC_SIZE)
Xf_test = Xf[N_TRAIN:].reshape(N_TEST, PIC_SIZE)
Yf_train=np.zeros((N_TRAIN))
Yf_test=np.zeros((N_TEST))

X_train = np.concatenate((Xt_train, Xf_train),axis=0)
Y_train = np.concatenate((Yt_train, Yf_train),axis=0)

X_test = np.concatenate((Xt_test, Xf_test),axis=0)
Y_test = np.concatenate((Yt_test, Yf_test),axis=0).astype(int)

In [313]:
X_train /= 16
X_test /= 16

In [314]:
model = svm.SVC(gamma='scale', probability=True)#,kernel="poly")
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)
Y_prob = model.predict_proba(X_test)

In [318]:
def kf_valid(Xf,Xt, reg, kernel, degree=3):
    X = np.concatenate((Xf,Xt)).reshape(-1, PIC_SIZE)
    Y = np.concatenate((np.zeros(100),np.ones(100)))
    kf = sklearn.model_selection.KFold(10,True)
    FAR = 0
    FRR = 0
    ll = 0
    Fs = 0
    for i in kf.split(X, Y):
        X_tr = X[i[0]]
        Y_tr = Y[i[0]]
        X_te = X[i[1]].astype(int)
        Y_te = Y[i[1]].astype(int)
        model = svm.SVC(gamma='scale', probability=True, 
                        C=reg, kernel=kernel, degree=degree)
        model.fit(X_tr,Y_tr)
        Y_pred = model.predict(X_te).astype(int)
        Y_prob = model.predict_proba(X_te)
        ll += metrics.log_loss(Y_te,Y_prob)
        Fs += metrics.f1_score(Y_te,Y_pred)
        tp = np.sum(Y_pred & Y_te)
        tn = np.sum(np.logical_not(Y_pred) & np.logical_not(Y_te))
        fp = np.sum(Y_pred & np.logical_not(Y_te))
        fn = np.sum(np.logical_not(Y_pred) & Y_te)

        FAR += fp/np.sum(Y_te==0)
        FRR += fn/np.sum(Y_te==1)
    return FAR,FRR,ll,Fs

In [319]:
options={}
for i in tqdm.tqdm(kernels, position=0, leave=True):
    losses=[]
    for j in np.arange(0.1, 10, 0.1):
        losses.append(kf_valid(Xf, Xt, j, i[0], i[1]))
    losses=np.array(losses)
    options[i]=np.concatenate((np.min(losses[:,:3],axis=0),10-np.max(losses[:,3],keepdims=True)))

100%|██████████| 11/11 [01:49<00:00,  9.92s/it]


In [320]:
means = np.zeros(4)
stds = np.zeros(4)
m = []
keys = list(options.keys())
for i in keys:
    m.append(options[i])
m=np.array(m)
means = np.mean(m,axis=0)
stds = np.std(m,axis=0)
m=(m-means)/stds
keys[np.argmin(np.sum(m,axis=1,keepdims=True))]

C:\Users\valer\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


('linear', 3)

Лінійна SVM показала кращий середній резутьат по цим метрикам

In [323]:
losses=[]
for i in tqdm.tqdm(np.arange(0.01, 10, 0.01),position=0, leave=True):
    losses.append(kf_valid(Xf,Xt, i, keys[np.argmin(np.sum(m,axis=1,keepdims=True))][0],
                           keys[np.argmin(np.sum(m,axis=1,keepdims=True))][1]))
    
losses=np.array(losses)

means = np.mean(losses,axis=0)
stds = np.std(losses,axis=0)
norm_losses=(losses-means)/stds

best = np.argmin(np.sum(norm_losses,axis=1,keepdims=True))

model = svm.SVC(gamma='scale', probability=True, 
                C=np.arange(0.01, 10, 0.01)[best],
                kernel=keys[np.argmin(np.sum(m,axis=1,keepdims=True))][0],
                degree=keys[np.argmin(np.sum(m,axis=1,keepdims=True))][1]
               )

model.fit(X_train,Y_train)
Y_pred = model.predict(X_test).astype(int)
Y_prob = model.predict_proba(X_test)
ll = metrics.log_loss(Y_test,Y_prob)
Fs = metrics.f1_score(Y_test,Y_pred)
tp = np.sum(Y_pred & Y_test)
tn = np.sum(np.logical_not(Y_pred) & np.logical_not(Y_test))
fp = np.sum(Y_pred & np.logical_not(Y_test))
fn = np.sum(np.logical_not(Y_pred) & Y_test)
FAR = fp/np.sum(Y_test==0)
FRR = fn/np.sum(Y_test==1)

print("Кращий результат при регуляризації {}: FRR = {}, FAR = {}, log-loss = {}, F-score = {}".format(
    np.arange(0.01, 10, 0.01)[best], FRR, FAR, ll, Fs
))

100%|██████████| 999/999 [01:18<00:00, 12.71it/s]


Кращий результат при регуляризації 7.2299999999999995: FRR = 0.0, FAR = 0.03333333333333333, log-loss = 0.07079869392194241, F-score = 0.9836065573770492


In [344]:
noise = sc.norm.rvs(0,0.2**0.5, (N_TEST*2, PIC_SIZE))
X_test_noised = noise+X_test
# X_test_noised=(X_test_noised-np.min(X_test_noised))/(np.max(X_test_noised)-np.min(X_test_noised))
Y_pred = model.predict(X_test_noised).astype(int)
Y_prob = model.predict_proba(X_test_noised)
ll = metrics.log_loss(Y_test,Y_prob)
Fs = metrics.f1_score(Y_test,Y_pred)
tp = np.sum(Y_pred & Y_test)
tn = np.sum(np.logical_not(Y_pred) & np.logical_not(Y_test))
fp = np.sum(Y_pred & np.logical_not(Y_test))
fn = np.sum(np.logical_not(Y_pred) & Y_test)
FAR = fp/np.sum(Y_test==0)
FRR = fn/np.sum(Y_test==1)
print("Та ж сама модельна зашумлених данних: FRR = {}, FAR = {}, log-loss = {}, F-score = {}".format(
    FRR, FAR, ll, Fs
))

Та ж сама модельна зашумлених данних: FRR = 0.23333333333333334, FAR = 0.13333333333333333, log-loss = 0.57014711766603, F-score = 0.8070175438596491


In [347]:
losses=[]
for i in range(3,9,2):
    X_test_denoised = ndimage.median_filter(X_test_noised,size=i)
#     X_test_denoised=(X_test_denoised-np.min(X_test_denoised))/(np.max(X_test_denoised)-np.min(X_test_denoised))
    Y_pred = model.predict(X_test_denoised).astype(int)
    Y_prob = model.predict_proba(X_test_denoised)
    ll = metrics.log_loss(Y_test,Y_prob)
    Fs = metrics.f1_score(Y_test,Y_pred)
    tp = np.sum(Y_pred & Y_test)
    tn = np.sum(np.logical_not(Y_pred) & np.logical_not(Y_test))
    fp = np.sum(Y_pred & np.logical_not(Y_test))
    fn = np.sum(np.logical_not(Y_pred) & Y_test)
    FAR = fp/np.sum(Y_test==0)
    FRR = fn/np.sum(Y_test==1)
    losses.append([FRR,FAR,ll,Fs])
size=3
for i in losses:
    print("вікно розміром {}: FRR = {}, FAR = {}, log-loss = {}, F-score = {}".format(size,*i))
    size+=2

вікно розміром 3: FRR = 0.16666666666666666, FAR = 0.0, log-loss = 0.28820068675319, F-score = 0.9090909090909091
вікно розміром 5: FRR = 0.16666666666666666, FAR = 0.0, log-loss = 0.2918095799138572, F-score = 0.9090909090909091
вікно розміром 7: FRR = 0.36666666666666664, FAR = 0.0, log-loss = 0.44170541440356775, F-score = 0.7755102040816326


З результатов видно, що фільтр з розмвром вікна 3 спрацював краще за інші

In [381]:
cur = X[Y==0]
pick = np.random.choice(cur.shape[0], size=100, replace=False)
X_train = cur[pick][:N_TRAIN]
X_test = cur[pick][N_TRAIN:]
Y_train = np.ones(N_TRAIN)*0
Y_test = np.ones(N_TEST)*0
for i in data['target_names'][1:]:
    cur = X[Y==i]
    pick = np.random.choice(cur.shape[0], size=100, replace=False)
    X_train = np.concatenate((X_train,cur[pick][:N_TRAIN]))
    X_test = np.concatenate((X_test,cur[pick][N_TRAIN:]))
    Y_train = np.concatenate((Y_train,np.ones(N_TRAIN)*i))
    Y_test = np.concatenate((Y_test,np.ones(N_TEST)*i))
X_train=X_train.reshape(-1,PIC_SIZE)/16
X_test=X_test.reshape(-1,PIC_SIZE)/16
Y_train=Y_train.astype(int)
Y_test=Y_test.astype(int)

In [388]:
model = svm.SVC(gamma='scale', probability=True, 
                C=np.arange(0.01, 10, 0.01)[best],
                kernel=keys[np.argmin(np.sum(m,axis=1,keepdims=True))][0],
                degree=keys[np.argmin(np.sum(m,axis=1,keepdims=True))][1],
                decision_function_shape = 'ovr'
               )
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test).astype(int)
Y_prob = model.predict_proba(X_test)
ll = metrics.log_loss(Y_test,Y_prob)
Fs = metrics.f1_score(Y_test,Y_pred,average="weighted")
tp = np.sum(Y_pred & Y_test)
tn = np.sum(np.logical_not(Y_pred) & np.logical_not(Y_test))
fp = np.sum(Y_pred & np.logical_not(Y_test))
fn = np.sum(np.logical_not(Y_pred) & Y_test)
FAR = fp/np.sum(Y_test==0)
FRR = fn/np.sum(Y_test==1)
print("Багатокласовий класифікатор: FRR = {}, FAR = {}, log-loss = {}, F-score = {}".format(FRR, FAR, ll, Fs))

Багатокласовий класифікатор: FRR = 0.0, FAR = 0.0, log-loss = 0.16935017108572942, F-score = 0.9698895238678756


In [411]:
noise = sc.norm.rvs(0,0.2**0.5, (N_TEST*10, PIC_SIZE))
X_test_noised = noise+X_test

Y_pred = model.predict(X_test_noised).astype(int)
Y_prob = model.predict_proba(X_test_noised)
ll = metrics.log_loss(Y_test,Y_prob)
Fs = metrics.f1_score(Y_test,Y_pred,average="weighted")
tp = np.sum(Y_pred & Y_test)
tn = np.sum(np.logical_not(Y_pred) & np.logical_not(Y_test))
fp = np.sum(Y_pred & np.logical_not(Y_test))
fn = np.sum(np.logical_not(Y_pred) & Y_test)
FAR = fp/np.sum(Y_test==0)
FRR = fn/np.sum(Y_test==1)
print("Багатокласовий класифікатор з зашумленими данними: FRR = {}, FAR = {}, log-loss = {}, F-score = {}".format(FRR, FAR, ll, Fs))

Багатокласовий класифікатор з зашумленими данними: FRR = 0.1, FAR = 0.06666666666666667, log-loss = 0.8531948689190054, F-score = 0.6916514668615735


In [413]:
losses=[]
for i in range(3,9,2):
    X_test_denoised = ndimage.median_filter(X_test_noised,size=i)
#     X_test_denoised=(X_test_denoised-np.min(X_test_denoised))/(np.max(X_test_denoised)-np.min(X_test_denoised))
    Y_pred = model.predict(X_test_denoised).astype(int)
    Y_prob = model.predict_proba(X_test_denoised)
    ll = metrics.log_loss(Y_test,Y_prob)
    Fs = metrics.f1_score(Y_test,Y_pred, average="weighted")
    tp = np.sum(Y_pred & Y_test)
    tn = np.sum(np.logical_not(Y_pred) & np.logical_not(Y_test))
    fp = np.sum(Y_pred & np.logical_not(Y_test))
    fn = np.sum(np.logical_not(Y_pred) & Y_test)
    FAR = fp/np.sum(Y_test==0)
    FRR = fn/np.sum(Y_test==1)
    losses.append([FRR,FAR,ll,Fs])
size=3
for i in losses:
    print("вікно розміром {}: FRR = {}, FAR = {}, log-loss = {}, F-score = {}".format(size,*i))
    size+=2

C:\Users\valer\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\valer\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


вікно розміром 3: FRR = 0.0, FAR = 0.26666666666666666, log-loss = 0.8062156518766509, F-score = 0.7444899004878585
вікно розміром 5: FRR = 0.0, FAR = 0.16666666666666666, log-loss = 1.4573908421039072, F-score = 0.4451216308930037
вікно розміром 7: FRR = 0.0, FAR = 0.3333333333333333, log-loss = 1.715787809911739, F-score = 0.2075360652381829


Знов фільтр з розмвром вікна 3 спрацював краще за інші. Але вийшло навіть гірше, ніж було до видалення шумів. Схоже, що разом із шумом ми видалили корисну інформацію 